# MULTI-LAYER PERCEPTRON
## FOR FACE RECOGNITION

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("Packages loaded")

Packages loaded


# LOAD DATA

In [2]:
# LOAD DATA
cwd = os.getcwd()
loadpath = cwd + "/data.npz"
l = np.load(loadpath)
print (l.files)

# PARSE LOADED DATA
trainimg   = l['trainimg']
trainlabel = l['trainlabel']
testimg    = l['testimg']
testlabel  = l['testlabel']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d TRAIN IMAGES" % (ntrain))
print ("%d TEST IMAGES" % (ntest))
print ("%d DIMENSIONAL INPUT" % (dim))
print ("%d CLASSES" % (nclass))

['imgsize', 'trainimg', 'categories', 'trainlabel', 'testlabel', 'testimg']
10042 TRAIN IMAGES
2511 TEST IMAGES
2304 DIMENSIONAL INPUT
7 CLASSES


# DEFINE MODEL

In [3]:
tf.set_random_seed(0)

# MULTILAYER PERCEPTRON MODEL
n_input    = dim    # INPUT LAYER DIMENSION
n_hid1     = 512    # 1ST HIDDEN LAYER DIMENSION
n_hid2     = 256    # 2ND HIDDEN LATER DIMENSION
n_output   = nclass # OUTPUT LAYER DIMENSION
x = tf.placeholder("float", [None, dim]) 
y = tf.placeholder("float", [None, nclass]) 
std = 0.1 
W = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hid1], stddev=std)),
    'h2': tf.Variable(tf.random_normal([n_hid1, n_hid2], stddev=std)),
    'out': tf.Variable(tf.random_normal([n_hid2, n_output], stddev=std))
}
b = {
    'b1': tf.Variable(tf.random_normal([n_hid1])),
    'b2': tf.Variable(tf.random_normal([n_hid2])),
    'out': tf.Variable(tf.random_normal([n_output]))
}
def multilayer_perceptron(_x, _W, _b):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_x, _W['h1']), _b['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _W['h2']), _b['b2']))
    return tf.matmul(layer_2, _W['out']) + _b['out']
print ("MULTILAYER PERCEPTRON MODEL READY")

MULTILAYER PERCEPTRON MODEL READY


# DEFINE FUNCTIONS 

In [4]:
learning_rate = 0.001
pred = multilayer_perceptron(x, W, b)
ce_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
WEIGHT_DECAY_FACTOR = 0.001 
l2_loss = tf.add_n([tf.nn.l2_loss(v) 
            for v in tf.trainable_variables()])
cost = ce_cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, tf.float32))
init = tf.initialize_all_variables()
print ("FUNCTIONS READY")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
FUNCTIONS READY


# RUN

In [5]:
# PARAMETERS
batch_size      = 256
display_step    = 20
training_epochs = 400
num_batch = int(ntrain/batch_size)+1

session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# LAUNCH THE GRAPH
sess = tf.Session(config=session_conf)
sess.run(init)
for epoch in range(training_epochs):
    avg_cost = 0.
    randpermlist = np.random.permutation(ntrain)
    for i in range(num_batch): 
        randidx  = randpermlist[i*batch_size:min((i+1)*batch_size, ntrain-1)]
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        feeds    = {x: batch_xs, y: batch_ys}
        # OPTIMIZE VARIABLES
        sess.run(optm, feed_dict=feeds)
        # COMPUTE LOSS
        avg_cost += sess.run(cost, feed_dict=feeds)/num_batch

    # PRINT CURRENT STATUS
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        feeds = {x: trainimg, y: trainlabel}
        train_acc = sess.run(accr, feed_dict=feeds)
        feeds = {x: testimg, y: testlabel}
        test_acc = sess.run(accr, feed_dict=feeds)
        print (" TRAIN ACCURACY: %.3f" % (train_acc))
        print (" TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 000/400 cost: 10.313727379
 TRAIN ACCURACY: 0.210
 TEST ACCURACY: 0.217
Epoch: 020/400 cost: 4.126761895
 TRAIN ACCURACY: 0.476
 TEST ACCURACY: 0.371
Epoch: 040/400 cost: 2.468688923
 TRAIN ACCURACY: 0.713
 TEST ACCURACY: 0.511
Epoch: 060/400 cost: 1.626562846
 TRAIN ACCURACY: 0.863
 TEST ACCURACY: 0.605
Epoch: 080/400 cost: 1.175332123
 TRAIN ACCURACY: 0.837
 TEST ACCURACY: 0.585
Epoch: 100/400 cost: 0.976072341
 TRAIN ACCURACY: 0.960
 TEST ACCURACY: 0.688
Epoch: 120/400 cost: 1.043804206
 TRAIN ACCURACY: 0.891
 TEST ACCURACY: 0.628
Epoch: 140/400 cost: 0.773103517
 TRAIN ACCURACY: 0.979
 TEST ACCURACY: 0.699
Epoch: 160/400 cost: 0.615308864
 TRAIN ACCURACY: 0.997
 TEST ACCURACY: 0.709
Epoch: 180/400 cost: 0.580374952
 TRAIN ACCURACY: 0.989
 TEST ACCURACY: 0.709
Epoch: 200/400 cost: 0.599490422
 TRAIN ACCURACY: 0.965
 TEST ACCURACY: 0.682
Epoch: 220/400 cost: 0.552516124
 TRAIN ACCURACY: 0.960
 TEST ACCURACY: 0.679
Epoch: 240/400 cost: 0.466565380
 TRAIN ACCURACY: 0.997
 TEST A